In [2]:
from edgar import get_filings
from edgar import *
from pandas import *
print("EdgarTools installed successfully!")

EdgarTools installed successfully!


c:\Users\shrey\Desktop\ScalerField-Quant\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from edgar import set_identity

# Use your name and email
# set_identity("John Doe john.doe@company.com")

# Or just your email
set_identity("shreybansal1165@gmail.com")

In [4]:
company = Company("AAPL")
latest_10k = company.get_filings(form="10-K").latest()
# print(latest_10k.text())
financials = latest_10k.sections()
count=0
count_big=0
for section in financials:
    if len(section)==0:
        pass
    count_big+=1

    if section.startswith("Item"):
        # print(section)
        count+=1

        # print("newsection start\n")
# Save the 10-K text to a file

# text_content = latest_10k.text()

# # Save to a text file
# with open("AAPL_latest_10K.txt", "w", encoding="utf-8") as f:
#     f.write(text_content)

# print(f"10-K text saved to AAPL_latest_10K.txt")
# print(f"File size: {len(text_content)} characters")
    

In [5]:
company = Company("AAPL")
latest_10k = company.get_filings(form="10-K").latest()
# print(latest_10k.text())
financials = latest_10k.sections()
for section in financials:
    if len(section)==0:
        pass

    if section.startswith("Item"):
        print(section)
        print("-------------------------------------------------------")

Item 1.    Business
Company Background
The Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services. The Company’s fiscal year is the 52- or 53-week period that ends on the last Saturday of September.
Products
iPhone
iPhone® is the Company’s line of smartphones based on its iOS operating system. The iPhone line includes iPhone 16 Pro, iPhone 16, iPhone 15, iPhone 14 and iPhone SE®.
-------------------------------------------------------
Item 1A.    Risk Factors
The Company’s business, reputation, results of operations, financial condition and stock price can be affected by a number of factors, whether currently known or unknown, including those described below. When any one or more of these risks materialize from time to time, the Company’s business, reputation, results of operations, financial condition and stock price can be materially and adversely affected.
Because of the following factors

In [ ]:
# Save the 10-K text to a file
text_content = latest_10k.text()

# Save to a text file
with open("AAPL_latest_10K.txt", "w", encoding="utf-8") as f:
    f.write(text_content)

print(f"10-K text saved to AAPL_latest_10K.txt")
print(f"File size: {len(text_content)} characters")

In [1]:
import os
import re

# Create directory for Item sections
output_dir = "AAPL_10K_Items"
os.makedirs(output_dir, exist_ok=True)

company = Company("AAPL")
latest_10k = company.get_filings(form="10-K").latest()

print(f"Processing 10-K filing from {latest_10k.filing_date}")
print(f"Accession: {latest_10k.accession_no}")
print(f"Output directory: {output_dir}")
print("-" * 50)

# Get all sections
sections = latest_10k.sections()
item_count = 0

for section_name in sections:
    # Check if section starts with "Item"
    if section_name.strip().startswith("Item"):
        try:
            # Extract item number (e.g., "Item 1", "Item 1A", etc.)
            item_match = re.match(r'Item\s+([0-9A-Z]+)', section_name, re.IGNORECASE)
            
            if item_match:
                item_number = item_match.group(1)
                
                # Get the text content for this section
                section_results = latest_10k.search(section_name)
                
                if section_results and len(section_results) > 0:
                    # Extract text from DocSection objects
                    text_parts = []
                    for result in section_results:
                        if hasattr(result, 'text'):
                            text_parts.append(result.text)
                        elif isinstance(result, str):
                            text_parts.append(result)
                        else:
                            # Try to convert to string
                            text_parts.append(str(result))
                    
                    # Join all text parts
                    full_text = "\n\n".join(text_parts)
                    
                    # Skip if content is too short (likely not the actual section)
                    if len(full_text.strip()) < 100:
                        print(f"⚠ Skipping {section_name} - content too short ({len(full_text)} chars)")
                        continue
                    
                    # Create filename (sanitize item number for filename)
                    safe_item_number = re.sub(r'[^A-Za-z0-9]', '', item_number)
                    filename = f"Item_{safe_item_number}.txt"
                    filepath = os.path.join(output_dir, filename)
                    
                    # Write section content to file
                    with open(filepath, 'w', encoding='utf-8') as f:
                        f.write(f"SECTION: {section_name}\n")
                        f.write(f"FILING DATE: {latest_10k.filing_date}\n")
                        f.write(f"COMPANY: Apple Inc. (AAPL)\n")
                        f.write(f"ACCESSION: {latest_10k.accession_no}\n")
                        f.write("="*80 + "\n\n")
                        f.write(full_text)
                    
                    item_count += 1
                    print(f"✓ Created: {filename} ({len(full_text):,} characters)")
                else:
                    print(f"⚠ No content found for: {section_name}")
            else:
                print(f"⚠ Could not parse item number from: {section_name}")
                
        except Exception as e:
            print(f"✗ Error processing {section_name}: {str(e)}")
            continue

print("-" * 50)
print(f"✅ Processing complete!")
print(f"📁 Directory: {output_dir}")
print(f"📄 Files created: {item_count}")

# List all created files
if item_count > 0:
    print(f"\nCreated files:")
    for filename in sorted(os.listdir(output_dir)):
        if filename.endswith('.txt'):
            filepath = os.path.join(output_dir, filename)
            file_size = os.path.getsize(filepath)
            print(f"  - {filename} ({file_size:,} bytes)")
else:
    print("\n⚠ No files were created. Let's try an alternative approach...")
    
    # Alternative approach: Get section content directly
    print("\nTrying alternative method...")
    
    for section_name in sections[:5]:  # Try first 5 sections
        if section_name.strip().startswith("Item"):
            print(f"\nTesting section: {section_name}")
            
            # Try different approaches to get content
            try:
                # Method 1: Try to get section object directly
                section_obj = latest_10k.obj()
                print(f"  - Section object type: {type(section_obj)}")
                
                # Method 2: Try full text search
                full_text = latest_10k.text()
                if section_name in full_text:
                    print(f"  - Found section in full text")
                else:
                    print(f"  - Section not found in full text")
                    
            except Exception as e:
                print(f"  - Error in alternative method: {str(e)}")

NameError: name 'Company' is not defined

In [6]:
tenk = latest_10k.obj()
print(tenk.doc)


╭───────────────────────────────────────────────── HTML Document ─────────────────────────────────────────────────╮
│                                                                                                                 │
│   Chunks   Items                                                                                     Avg Size   │
│  ─────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   300      Item 1,Item 1A,Item 1B,Item 1C,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 7A,Item     876        │
│            8,Item 9,Item 9A,Item 9B,Item 9C,Item 10,Item 11,Item 12,Item 13,Item 14,Item 15,Item                │
│            16                                                                                                   │
│                                                                                                                 │
╰───────────────────────────────────────────────────────────────────────

In [8]:
facts = company.get_facts()
if facts:
    # Convert to pandas DataFrame
    facts_df = facts.to_pandas()

    # Get number of facts
    num_facts = facts.num_facts()
    print(f"Company has {num_facts} XBRL facts")

AttributeError: 'EntityFacts' object has no attribute 'to_pandas'